In [14]:
import pandas as pd
import savReaderWriter as spss
from pathlib import Path

In [15]:
df = pd.read_csv('Data files/PISA data/stuscho.csv')
country_dict_df = pd.read_csv('Data files/PISA data/nations.csv')

C:\Users\RobinQu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20,22,23,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267

In [16]:
def sav_to_dataframe(file):
    if not os.path.isfile(file):
        print('File not exists!')
        raise FileNotFoundError()
    
    if not file.endswith('.sav'):
        print('It is not a sav file!')
        raise FileNotFoundError()

    records = []
    with spss.SavReader(file) as reader:
        print("Reading file:",file, "...")
        for line in reader:
            records.append(line)

    df = pd.DataFrame(records)
    return df

In [17]:
def student2school(df, id):
    '''
        Takes the 'IDs_sorted_by_student.csv' dataframe and student id, returns school id of the student.
        '''
    return student2schoolHelper(df, id, 0, len(df))

In [18]:
def student2schoolHelper(df, id, start, end):
    if start > end:
        print('Student ID:' + id + ' not exists!')
        raise KeyError()
    elif start == end:
        if id == student2schoolHelper['CNTSCHID'][start]:
            return student2schoolHelper['SUBNATIO'][start]
        else:
            print('Student ID:' + id + ' not exists!')
            raise KeyError()
    index = (end + start) // 2
    mid_value = df['CNTSCHID'][index]
    if mid_value == id:
        return df['SUBNATIO'][index]
    elif mid_value > id:
        return student2schoolHelper(df, id, start, index)
    else:
        return student2schoolHelper(df, id, index, end)

In [19]:
def school2nation(id):
    '''
        Takes school id, returns nation id of the school.
        '''
    if id < 97100000:
        return id // 100000 * 10000
    if id < 97200000:
        return 7240000
    return 8400000

In [20]:
def student2nation(id):
    '''
        Takes student id, returns nation id of the student.
        '''
    return school2nation(id)

In [21]:
def info(df, id):
    '''
    :param df: the dataframe of 'nations.csv'
    :param id: the id of nation
    :return: a tuple - (string country name, string continent)
    '''
    try:
        target = df[df['nationID'] == id]
    except KeyError:
        print('Nation ID:' + id + ' not exists!')
        raise KeyError()

    name = target['nationName'].tolist()
    continent = target.continent.tolist()

    return (name[0],continent[0])

In [22]:
def infos(country_df, id_list):
    '''
    :param country_df: the dataframe of 'nations.csv'
    :param id_df: the dataframe of country id
    :return: a dataframe - (string country name, string continent)
    '''

    name_list = []
    continent_list = []

    for i in range(len(id_list)):
        t = info(country_df, id_list[i])
        name_list.append(t[0])
        continent_list.append(t[1])

    dict = {'CountryName':name_list, 'Continent':continent_list}
    res = pd.DataFrame(dict)

    return res

In [23]:
df1 = df[['CNTRYID', 'CNTSCHID', 'CNTSTUID', 'ST004D01T']]
df1.columns = ['CountryID', 'SchoolID', 'StudentID', 'Gender']
stuid_list = df1['StudentID'].tolist()
countryid_list = []
for stuid in stuid_list:
    countryid_list.append(student2nation(stuid))
df_country = infos(country_dict_df, countryid_list)
df_math = df.iloc[:, 809:819]
df_math['Mathematics'] = df_math.mean(axis=1)
df_read = df.iloc[:, 819:829]
df_read['Reading'] = df_read.mean(axis=1)
df_science = df.iloc[:, 829:839]
df_science['Science'] = df_science.mean(axis=1)
df_hlm = df[['IBTEACH', 'WEALTH', 'ESCS', 'SC013Q01TA', 'SCIERES']]

In [24]:
output_df = pd.concat([df1, df_country, df_math['Mathematics'],
                       df_read['Reading'], df_science['Science'],
                       df_hlm], axis=1, sort=False)
output_df.loc[df1['Gender'] == 1, 'Gender'] = 'Female'
output_df.loc[df1['Gender'] == 2, 'Gender'] = 'Male'
output_df.to_csv('student_info.csv', encoding='utf-8')